<a href="https://colab.research.google.com/github/nguyenvuong1309/Do-an-lap-trinh-an-toan/blob/main/BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import pandas as pd
from numpy import zeros, newaxis
import numpy as np
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional,Activation



In [29]:
csv_file_path = '/content/drive/MyDrive/Colab/dataset/text_dataset.csv'
df = pd.read_csv(csv_file_path)

In [30]:
dataset = df

In [31]:
dataset = dataset[['1','label']]
dataset.head()

,1,label
0,/dirent_uri.c memset \nconst char *svn_relpat...,0
1,/CWE_Uncontrolled_Format_String__char_environ...,0
2,/CWE_Cleartext_Tx_Sensitive_Info__w_wchar_t_c...,0
3,/utils.c strcmp \nint avformat_open_input(AVF...,0
4,/CWE_Cleartext_Tx_Sensitive_Info__w_wchar_t_l...,32


In [32]:
print(dataset['label'].unique())
print(len(dataset['label'].unique()))

[ 0 32  4 10 20 36  3  1 12 25 15 34 16 11  2 14 35 38  9 29  7 18 37 21
 22 23  5  8 17 31 39 33 28 30 19 40 26 13]
38


In [33]:
train_dataset = dataset[0:int(len(dataset) * 0.8)]
test_dataset =  dataset[int(len(dataset) * 0.8) :]

In [34]:
#<_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>
text_data = train_dataset['1'].values
tensor = tf.data.Dataset.from_tensor_slices(text_data)

In [35]:
tensor

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [36]:
train_dataset_numpy = np.array(train_dataset)

In [37]:
print(train_dataset_numpy[:3])

[[' /dirent_uri.c memset \nconst char *svn_relpath__internal_style(const char *relpath,apr_pool_t *pool) \nreturn svn_relpath_canonicalize ( internal_style ( relpath , pool ) , pool ) ; \nconst char *svn_relpath_canonicalize(const char *relpath,apr_pool_t *pool) \nreturn canonicalize ( type_relpath , relpath , pool ) ; \nstatic const char *canonicalize(path_type_t type,const char *path,apr_pool_t *pool) \nchar * canon ; \nchar * dst ; \nif ( path [  ] == \'\\\' )  \ndst = canon = ( memset ( apr_palloc ( pool , strlen ( path ) +  ) ,  , strlen ( path ) +  ) ); \nwhile ( ( * src ) != \':\' )  \n* ( dst ++ ) = canonicalize_to_lower ( * ( src ++ ) ); \n* ( dst ++ ) = \':\'; \n* ( dst ++ ) = ; \n* ( dst ++ ) = ; \nsrc += ; \nseg = src; \nwhile ( * src && ( * src ) !=  && ( * src ) !=  )  \nsrc ++; \nif ( ( * src ) ==  )  \nseglen = ( src - seg +  ); \nmemcpy ( dst , seg , seglen ); \ndst += seglen; \nsrc ++; \nsrc = seg; \nif ( ( * src ) == \'[\' )  \n* ( dst ++ ) = * ( src ++ ); \nwhile ( 

In [38]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [39]:
train_dataset = train_dataset.sample(frac=0.5, replace=True, random_state=1)
test_dataset = test_dataset.sample(frac=0.5, replace=True, random_state=1)

In [40]:
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(tensor)

In [41]:
train_dataset['1']

5157     /CWE_Heap_Based_Buffer_Overflow__cpp_CWE_char...
235      /CWE_Trapdoor__ip_based_logic_.c send \nvoid ...
3980     /CWE_OS_Command_Injection__char_console_wspaw...
5192     /CWE_Uncontrolled_Format_String__wchar_t_cons...
7935     /CWE_Uncontrolled_Mem_Alloc__new_wchar_t_list...
                              ...                        
5934     /CWE_Unexpected_Sign_Extension__listen_socket...
5286     /config.c strlen \nsvn_error_t *svn_config_ge...
3947     /CWE_Integer_Overflow__int_connect_socket_squ...
3673     /CWE_Heap_Based_Buffer_Overflow__cpp_CWE_wcha...
4918     /CWE_Cleartext_Tx_Sensitive_Info__w_char_conn...
Name: 1, Length: 4000, dtype: object

In [42]:
train_dataset

,1,label
5157,/CWE_Heap_Based_Buffer_Overflow__cpp_CWE_char...,3
235,/CWE_Trapdoor__ip_based_logic_.c send \nvoid ...,34
3980,/CWE_OS_Command_Injection__char_console_wspaw...,0
5192,/CWE_Uncontrolled_Format_String__wchar_t_cons...,0
7935,/CWE_Uncontrolled_Mem_Alloc__new_wchar_t_list...,0
...,...,...
5934,/CWE_Unexpected_Sign_Extension__listen_socket...,0
5286,/config.c strlen \nsvn_error_t *svn_config_ge...,0
3947,/CWE_Integer_Overflow__int_connect_socket_squ...,0
3673,/CWE_Heap_Based_Buffer_Overflow__cpp_CWE_wcha...,0


In [43]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'if', 'data', 'char', 'void', 'i', 'd', 's', 'int',
       'service', 'sizeof', 'recvresult', 'static', 'datalen', 'null',
       'return', 'line', 'wchart', 'databuffer'], dtype='<U223')

In [44]:
example = [[' /CWE_Heap_Based_Buffer_Overflow__cpp_CWE_char_snprintf_.cpp memset \nvoid bad() \nchar * data ; \ndata = new char [  ]; \nif ( GLOBAL_CONST_TRUE )  \nmemset ( data , \'A\' ,  -  ); \ndata [  -  ] = \'\\\'; \nSNPRINTF ( dest , strlen ( data ) , "%s" , data ); \nprintLine ( data ); \nvoid printLine (const char * line) \nif ( line != NULL )  \nprintf ( "%s\\n" , line ); \ndelete [ ] data \n\n'],
 [' /CWE_Trapdoor__ip_based_logic_.c send \nvoid CWE_Trapdoor__ip_based_logic__bad() \nWSADATA wsaData ; \nstruct sockaddr_in service , acceptService ; \nif ( WSAStartup ( MAKEWORD (  ,  ) , & wsaData ) != NO_ERROR )  \nlistenSocket = socket ( AF_INET , SOCK_STREAM , IPPROTO_TCP ); \nif ( listenSocket == INVALID_SOCKET )  \nmemset ( & service ,  , sizeof ( service ) ); \nservice . sin_family = AF_INET; \nservice . sin_addr . s_addr = INADDR_ANY; \nservice . sin_port = htons (  ); \nif ( bind ( listenSocket , ( struct sockaddr * ) & service , sizeof ( service ) ) == SOCKET_ERROR )  \nif ( listen ( listenSocket ,  ) == SOCKET_ERROR )  \nacceptSocket = accept ( listenSocket , NULL , NULL ); \nif ( acceptSocket == SOCKET_ERROR )  \nif ( getsockname ( acceptSocket , ( struct sockaddr * ) & acceptService , & acceptServiceLen ) == -  )  \nif ( strcmp ( "..." , inet_ntoa ( acceptService . sin_addr ) ) ==  )  \nif ( send ( acceptSocket , ADMIN_MESSAGE , strlen ( ADMIN_MESSAGE ) ,  ) == SOCKET_ERROR )  \nif ( send ( acceptSocket , DEFAULT_MESSAGE , strlen ( DEFAULT_MESSAGE ) ,  ) == SOCKET_ERROR )  \nwhile (  )  \n\n'],
 [' /CWE_OS_Command_Injection__char_console_wspawnl_.c strlen \nvoid CWE_OS_Command_Injection__char_console_wspawnl__bad() \nchar * data ; \nchar dataBuffer [  ] = "" ; \ndata = dataBuffer; \nif ( staticFive ==  )  \nsize_t dataLen = strlen ( data ) ; \nif (  - dataLen >  )  \nif ( fgets ( data + dataLen , ( int ) (  - dataLen ) , stdin ) != NULL )  \ndataLen = strlen ( data ); \nif ( dataLen >  && data [ dataLen -  ] == \'\\n\' )  \ndata [ dataLen -  ] = \'\\\'; \n\n']
           ]

In [45]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[8907,   29,    5,  139,    4,    3,    3,  195,    4,    2,  592,
          29,    3,  102,    3,  235,   36,   23,    3,    8,    3,   46,
           3,    5,   46,   30,    4,   17,    2,   17,   15,   59,   80,
          17,  283,    3,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [3333, 1218,    5, 5299,   26,   26,   21,   84,   10, 2053,    2,
          86,   88,   26,   93,   28,   62,   44,   87,   92,    2,   28,
          54,   29,   10,   11,   10,   10,   90,   44,   10,   83,   85,
         156,   10,   89,   91,    2,  153,   28,   21,   81,   10,   11,
          10,   24,    2,  154,   28,   24,   45,  155,   28,   15,   15,
           2,   45,   24,    2, 3826,   45,   21,   81,

In [46]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=1,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(41, activation='softmax')
])

In [47]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [48]:
X_train = train_dataset['1']
y_train = train_dataset['label']
y_train = tf.keras.utils.to_categorical(y_train, num_classes=41)

In [49]:
print(X_train.shape)
print(y_train.shape)

(4000,)
(4000, 41)


In [50]:
history = model.fit(X_train,y_train, epochs=1,
                    # validation_data=test_dataset,
                    # validation_steps=30
                    )

125/125 [==============================] - 153s 1s/step - loss: 2.8657 - accuracy: 0.7480


In [51]:
X_test = test_dataset['1']
y_test = test_dataset['label']
y_test = tf.keras.utils.to_categorical(y_test, num_classes=41)

In [52]:
test_loss, test_acc = model.evaluate(X_test,y_test)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

32/32 [==============================] - 13s 264ms/step - loss: 1.2689 - accuracy: 0.7800
Test Loss: 1.2688937187194824
Test Accuracy: 0.7799999713897705
